##### データセットの場所やバッチサイズなどの定数値の設定

In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


# 使用するデバイス
# Paperspace Gradient などで GPU を利用する場合は DEVICE = 'cuda:0' とする
DEVICE = 'cpu'

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
N_EPOCHS = 10

# 学習時のバッチサイズ
BATCH_SIZE = 100

# 訓練データセット（画像ファイルリスト）のファイル名
TRAIN_DATASET_CSV = './MNIST/train_list.csv'

# テストデータセット（画像ファイルリスト）のファイル名
TEST_DATASET_CSV = './MNIST/test_list.csv'

# 画像ファイルの先頭に付加する文字列（データセットが存在するディレクトリのパス）
DATA_DIR = './MNIST'

# 学習結果の保存先フォルダ
MODEL_DIR = './CNN_models'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE = os.path.join(MODEL_DIR, 'mnist_recognizer_model.pth')

##### ニューラルネットワークモデルの定義

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# MNIST画像認識AIを実現するニューラルネットワーク
# 畳込みニューラルネットワーク（CNN）のサンプル
class MNISTRecognizer(nn.Module):

    def __init__(self):
        super(MNISTRecognizer, self).__init__()

        # 畳込み層1: 入力（1チャンネル画像）→ 1層目（4チャンネル特徴マップ），カーネルサイズ3x3，ストライド幅1，パディングなし
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding=0)

        # max-pooling層: 特徴マップの縦幅・横幅をそれぞれ 1/2 に（元々の幅が奇数の場合は端数切捨て）
        self.pool = nn.MaxPool2d(kernel_size=2)

        # 畳込み層2: 1層目（4チャンネル特徴マップ）→ 2層目（4チャンネル特徴マップ），カーネルサイズ3x3，ストライド幅1，パディングなし
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=4, kernel_size=3, stride=1, padding=0)

        # 平坦化: 3次元の特徴マップを1次元（一列）に並べ直す
        self.flat = nn.Flatten()

        # 全結合層: 2層目（4チャンネル特徴マップ，パーセプトロン数は4x11x11）→ 出力（10次元）
        self.fc = nn.Linear(in_features=4*11*11, out_features=10)

    def forward(self, x):
        h = self.conv1(x) # 畳込み層1に画像を入力，画像サイズが 1x28x28 なので，特徴マップのサイズは 4x26x26 になる
        h = F.relu(h) # 活性化関数 ReLU
        h = self.pool(h) # max-pooling層を適用する．適用前の特徴マップのサイズが 4x26x26 なので，適用後は 4x13x13 になる
        h = self.conv2(h) # 続いて畳込み層2に通す．これにより特徴マップのサイズは 4x13x13 から 4x11x11 に変わる
        h = F.relu(h) # 活性化関数 ReLU
        h = self.flat(h) # 平坦化する．これにより 4x11x11=484 個のパーセプトロンが一列に並んだ「層」となる
        y = self.fc(h) # 最後に全結合層に通す
        return y

##### 訓練データセットの読み込み

In [3]:
import pickle
from torch.utils.data import DataLoader, random_split
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, 訓練データセットを用意
dataset = CSVBasedDataset(
    filename = TRAIN_DATASET_CSV,
    items = [
        'File Path', # X
        'Class Label' # Y
    ],
    dtypes = [
        'image', # Xの型
        'label' # Yの型
    ],
    dirname = DATA_DIR
)
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'wb') as fdicts_file:
    pickle.dump(dataset.forward_dicts, fdicts_file)

# 認識対象のクラス数を取得
n_classes = len(dataset.forward_dicts[1])

# 訓練データセットを分割し，一方を検証用に回す
dataset_size = len(dataset)
valid_size = int(0.05 * dataset_size) # 全体の 5% を検証用に
train_size = dataset_size - valid_size # 残りの 95% を学習用に
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# 訓練データおよび検証用データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習処理の実行

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.visualizers import LossVisualizer


# ニューラルネットワークの作成
model = MNISTRecognizer().to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
optimizer = optim.Adam(model.parameters())

# 損失関数：クロスエントロピー損失を使用
loss_func =  nn.CrossEntropyLoss()

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['train loss', 'valid loss'])

# 勾配降下法による繰り返し学習
for epoch in range(N_EPOCHS):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    model.train()
    sum_loss = 0
    for X, Y in tqdm(train_dataloader): # X, Y は CSVBasedDataset クラスの __getitem__ 関数の戻り値に対応
        for param in model.parameters():
            param.grad = None
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X) # 入力画像 X を現在のニューラルネットワークに入力し，出力の推定値を得る
        loss = loss_func(Y_pred, Y) # 損失関数の現在値を計算
        loss.backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
        optimizer.step() # 勾配に沿ってパラメータの値を更新
        sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 訓練データに対する損失関数の値を記録
    print('train loss = {0:.6f}'.format(avg_loss))

    # 検証
    model.eval()
    sum_loss = 0
    n_failed = 0
    with torch.inference_mode():
        for X, Y in tqdm(valid_dataloader):
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)
            Y_pred = model(X)
            loss = loss_func(Y_pred, Y)
            sum_loss += float(loss) * len(X)
            n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    avg_loss = sum_loss / valid_size
    loss_viz.add_value('valid loss', avg_loss) # 検証用データに対する損失関数の値を記録
    accuracy = (valid_size - n_failed) / valid_size
    print('valid loss = {0:.6f}'.format(avg_loss))
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')

# 学習結果のニューラルネットワークモデルをファイルに保存
model = model.to('cpu')
torch.save(model.state_dict(), MODEL_FILE)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### テストデータセットの読み込み

In [5]:
import pickle
from torch.utils.data import DataLoader
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, テストデータセットを用意
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'rb') as fdicts_file:
    fdicts = pickle.load(fdicts_file)
test_dataset = CSVBasedDataset(
    filename = TEST_DATASET_CSV,
    items = [
        'File Path', # X
        'Class Label' # Y
    ],
    dtypes = [
        'image', # Xの型
        'label' # Yの型
    ],
    dirname = DATA_DIR,
    fdicts = fdicts
)
test_size = len(test_dataset)

# 認識対象のクラス数を取得
n_classes = len(test_dataset.forward_dicts[1])

# テストデータをミニバッチに分けて使用するための「データローダ」を用意
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習済みニューラルネットワークモデルのロード

In [6]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
model = MNISTRecognizer()
model.load_state_dict(torch.load(MODEL_FILE))

##### 単一画像に対するテスト処理の実行

In [ ]:
import torch
from mylib.data_io import show_single_image


model = model.to(DEVICE)
model.eval()

# index 番目のテストデータをニューラルネットワークに入力してみる
while True:
    print('index?: ', end='')
    val = input()
    if val == 'exit': # 'exit' とタイプされたら終了
        break
    index = int(val)
    x, y = test_dataset[index]
    x = x.reshape(1, *x.size()).to(DEVICE)
    with torch.inference_mode():
        y_pred = model(x)
    y_pred = torch.argmax(y_pred, dim=1)
    print('')
    print('estimated:', int(y_pred))
    print('ground truth:', int(y))
    print('')
    show_single_image(x.to('cpu'), title='input image', sec=1)

##### 全ての画像に対するテスト処理の実行

In [ ]:
import torch
from tqdm import tqdm


model = model.to(DEVICE)
model.eval()

# テストデータセットを用いて認識精度を評価
n_failed = 0
with torch.inference_mode():
    for X, Y in tqdm(test_dataloader):
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X)
        n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    accuracy = (test_size - n_failed) / test_size
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')